<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [19]:
# librerias

import numpy as np
import pandas as pd

import scipy.stats as stats
import statsmodels.stats.api as sms

import pylab as plt
import seaborn as sns

from scipy.stats import bernoulli, beta

from scipy.stats import beta
%pip install bayes
from bayes import *

%matplotlib inline

  Created wheel for bayes: filename=bayes-0.1.1-py3-none-any.whl size=5533 sha256=d5ec0f1aa2d28a1620526bdba677df8c17d3ff4626bd5d50afd79b40daaf5fbe
  Stored in directory: /Users/alonsoherrera/Library/Caches/pip/wheels/8b/e1/33/bd265d7768a8f78d2ca909f6a70c9bfb334623942e3709cf3b
Successfully built bayes
Note: you may need to restart the kernel to use updated packages.


In [32]:
# datos
data=pd.read_csv('data/cookie_cats.csv')

data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [33]:
# transformacion

data.version=np.where(data.version=='gate_30', 'A', 'B')

data.retention_1=np.where(data.retention_1==True, 1, 0)
data.retention_7=np.where(data.retention_7==True, 1, 0)

data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,A,3,0,0
1,337,A,38,1,0
2,377,B,165,1,0
3,483,B,1,0,0
4,488,B,179,1,1


In [34]:
control=data[data.version=='A'].sample(n=44000, random_state=42)

tratamiento=data[data.version=='B'].sample(n=44000, random_state=42)

ab_test=pd.concat([control, tratamiento], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,6531033,A,35,0,0
1,8003009,A,4,0,0
2,5566807,A,23,1,0
3,64235,A,1,0,0
4,6210551,A,7,0,0


In [35]:
imps_ctrl, convs_ctrl= len(control), sum(control.retention_1)
imps_test, convs_test= len(tratamiento), sum(tratamiento.retention_1)

CR_ctrl=convs_ctrl/imps_ctrl
CR_test=convs_test/imps_test

print ('Tasas', CR_ctrl, CR_test)

Tasas 0.4484318181818182 0.44279545454545455


In [37]:
a_C, b_C = convs_ctrl+1, imps_ctrl-convs_ctrl+1
a_T, b_T = convs_test+1, imps_test-convs_test+1

beta_C=beta(a_C, b_C)  
beta_T=beta(a_T, b_T)

In [38]:


salto=(beta_T.mean()-beta_C.mean())/beta_C.mean()

In [39]:
prob=calc_prob(beta_T, beta_C)

NameError: name 'calc_prob' is not defined